<a href="https://colab.research.google.com/github/Shafqatsarwar/Google_colab_langchain_gemini/blob/main/crewai_crews_advance_concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

from google.colab import userdata

os.environ['GEMINI_API_KEY'] =  userdata.get('GEMINI_API_KEY')

In [ ]:
!pip install -Uq crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.6 MB/s 

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from crewai.flow.flow import Flow, start, listen


class MyFlow(Flow):

  @start()
  def function1(self):
    print("step1")

  @listen(function1)
  def function2(self):
    print("step2")

obj = MyFlow()
obj.kickoff()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: MyFlow                                                                                                   │
│  ID: 0674b22b-10d3-4a3e-8714-053544def95a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🌊 Flow: MyFlow
    ID: 0674b22b-10d3-4a3e-8714-053544def95a
└── 🧠 Starting Flow...

 Flow started with ID: 0674b22b-10d3-4a3e-8714-053544def95a


🌊 Flow: MyFlow
    ID: 0674b22b-10d3-4a3e-8714-053544def95a
├── 🧠 Starting Flow...
└── 🔄 Running: function1

step1


🌊 Flow: MyFlow
    ID: 0674b22b-10d3-4a3e-8714-053544def95a
├── Flow Method Step
└── ✅ Completed: function1

🌊 Flow: MyFlow
    ID: 0674b22b-10d3-4a3e-8714-053544def95a
├── Flow Method Step
├── ✅ Completed: function1
└── 🔄 Running: function2

step2


🌊 Flow: MyFlow
    ID: 0674b22b-10d3-4a3e-8714-053544def95a
├── Flow Method Step
├── ✅ Completed: function1
└── ✅ Completed: function2

✅ Flow Finished: MyFlow
├── Flow Method Step
├── ✅ Completed: function1
└── ✅ Completed: function2

In [ ]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

llm1.call("What is the capital of Pakistan?")

'The capital of Pakistan is **Islamabad**.\n'

In [ ]:
google_embedder = {
    "provider": "google",
    "config": {
         "model": "models/text-embedding-004",
         "api_key": userdata.get('GEMINI_API_KEY'),
         }
}

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = "Users name is Khan Sarwar. He is 35 years old and lives in Lahore, Pakistan. He is working as AI Agent Developer at PIAIC Pakistan"
string_source = StringKnowledgeSource(
    content=content,
)


# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    memory=True,
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
    embedder=google_embedder

)

result = crew.kickoff(inputs={"question": "What city does Khan Sarwar live in and how old is he?"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 259921cb-9e16-429e-9a82-d61ad5da88b6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 863cdfd0-fb75-45d2-82c8-82b4d023ff4e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 863cdfd0-fb75-45d2-82c8-82b4d023ff4e
       Status: Executing Task...
    └── 🤖 Agent: About User
            Status: In Progress

# Agent: About User
## Task: Answer the following questions about the user: What city does Khan Sarwar live in and how old is he?


🚀 Crew: crew
└── 📋 Task: 863cdfd0-fb75-45d2-82c8-82b4d023ff4e
       Status: Executing Task...
    └── 🤖 Agent: About User
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: About User
## Final Answer: 
Khan Sarwar lives in Lahore, Pakistan, and he is 35 years old.




🚀 Crew: crew
└── 📋 Task: 863cdfd0-fb75-45d2-82c8-82b4d023ff4e
       Status: Executing Task...
    └── 🤖 Agent: About User
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 863cdfd0-fb75-45d2-82c8-82b4d023ff4e
       Assigned to: About User
       Status: ✅ Completed
    └── 🤖 Agent: About User
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 863cdfd0-fb75-45d2-82c8-82b4d023ff4e                                                                     │
│  Agent: About User                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 259921cb-9e16-429e-9a82-d61ad5da88b6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# New Section- Memory
 * Long-term, Short-term, Entity Memory
 * you can add path and folder in colab to set as Data base for meomry, call back when ever you need or reload and pullto your next project as Data



In [ ]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crew2'

In [ ]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

In [ ]:

from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
# from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List, Optional

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage1.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=google_embedder,
                type="short_term",
                path="./my_crew2/short_term1/"
            )
        ),

    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity1/"
        )
    ),
    verbose=True,
)


# Integrating Mem0 for Enhanced User Memory

In [ ]:
# !pip install --upgrade crewai

In [ ]:
# to test all packages avaialable print below
import crewai
import crewai.memory

# List contents of the 'memory' submodule
print(dir(crewai.memory))

['EntityMemory', 'LongTermMemory', 'ShortTermMemory', 'UserMemory', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'contextual', 'entity', 'long_term', 'memory', 'short_term', 'storage', 'user']


In [ ]:
# import os
# from crewai import Crew, Process
# from mem0 import MemoryClient

# # Set environment variables for Mem0
# os.environ["MEM0_API_KEY"] = userdata.get('MEM0_API_KEY')

# # Step 1: Record preferences based on past conversation or user input
# client = MemoryClient()
# # messages = [
# #     {"role": "user", "content": "Hi there! I'm planning a vacation and could use some advice."},
# #     {"role": "assistant", "content": "Hello! I'd be happy to help with your vacation planning. What kind of destination do you prefer?"},
# #     {"role": "user", "content": "I am more of a beach person than a mountain person."},
# #     {"role": "assistant", "content": "That's interesting. Do you like hotels or Airbnb?"},
# #     {"role": "user", "content": "I like Airbnb more."},
# # ]
# # client.add(messages, user_id="john")

# # Step 2: Create a Crew with User Memory

# # Create an agent with the knowledge store
# agent = Agent(
#     role="About User",
#     goal="You know everything about the user.",
#     backstory="""You are a master at understanding people and their preferences.""",
#     verbose=True,
#     allow_delegation=False,
#     llm=llm1,
# )
# task = Task(
#     description="Answer the following questions about the user: {question}",
#     expected_output="An answer to the question.",
#     agent=agent,
# )

# crew = Crew(
#     agents=[agent],
#     tasks=[task],
#     process = Process.sequential,
#     verbose=True,
#     memory=True,
#     memory_config={
#         "provider": "mem0",
#         "config": {"user_id": "john"},
#     },
# )

# crew.kickoff(inputs={"question": "What is your favorite vacation destination?"})


# Tool

In [ ]:
!pip install -Uq 'crewai[tools]'


In [ ]:
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')


In [ ]:
embedder = {
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }


In [ ]:
!pip install -Uq langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI

llm1=LLM(model="gemini/gemini-2.0-flash")

In [ ]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY') # serper.dev API key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the Electric cars industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True,
    llm=llm1
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the Electric cars industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True,
    llm=llm1
)

# Define tasks
research = Task(
    description='Research the latest trends in the Electric cars industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the Electric cars industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the Electric cars industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
    planning_llm=llm1
)

# Execute tasks
crew.kickoff()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_t

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 620fa725-12a4-4912-8275-06e6a6734cbb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 
[2025-03-26 21:08:00][INFO]: Planning the crew execution


🚀 Crew: crew
└── 📋 Task: 6e84bc24-9bc6-4e70-9a17-67f551269df2
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6e84bc24-9bc6-4e70-9a17-67f551269df2
       Status: Executing Task...
    └── 🤖 Agent: Task Execution Planner
            Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 6e84bc24-9bc6-4e70-9a17-67f551269df2
│      Assigned to: Task Execution Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Task Execution Planner
│           Status: ✅ Completed
├── 📋 Task: 71e28315-aa1c-4fd5-848c-93d25fec4e4b
│      Assigned to: Market Research Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Market Research Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (4)
└── 📋 Task: 4ca188ab-7374-4f8b-8c0e-1baa082aaf7f
       Status: Executing Task...
    └── 🤖 Agent: Content Writer
            Status: ✅ Completed
        ├── 🔧 Used List files in directory (1)
        └── 🔧 Used Read a file's content (2)

🚀 Crew: crew
├── 📋 Task: 6e84bc24-9bc6-4e70-9a17-67f551269df2
│      Assigned to: Task Execution Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Task Execution Planner
│           Status: ✅ Completed
├── 📋 Task: 71e28315-aa1c-4fd5-848c-93d25fec4e4b
│      Assigned to: Market Research Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Market Research Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (4)
└── 📋 Task: 4ca188ab-7374-4f8b-8c0e-1baa082aaf7f
       Assigned to: Content Writer
       Status: ✅ Completed
    └── 🤖 Agent: Content Writer
            Status: ✅ Completed
        ├── 🔧 Used List files in directory (1)
        └── 🔧 Used Read a file's content (2)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4ca188ab-7374-4f8b-8c0e-1baa082aaf7f                                                                     │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 620fa725-12a4-4912-8275-06e6a6734cbb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw="```markdown\n## Electric Cars: Charging Ahead into the Future\n\nThe electric car revolution is no longer a distant dream; it's happening right now! With each passing day, EVs are becoming more appealing and accessible to a wider audience. What's driving this electrifying transformation? Let's dive into the top three trends shaping the electric car industry and paving the way for a cleaner, greener future.\n\nOne of the most exciting developments is the rapid advancement in battery technology. Forget the limitations of yesterday's batteries! We're talking about next-generation batteries like solid-state and lithium-sulfur, promising longer ranges, faster charging times, and enhanced safety. These innovations directly address the common concerns of potential EV buyers, making electric cars a more practical and attractive option for daily commutes and long road trips alike.\n\nAnother crucial factor fueling the EV surge is the ever-expanding charging infrastructure. Imagi

In [ ]:
# prompt: import markdown for jupyter display

from IPython.display import Markdown

def display_markdown_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    display(Markdown(content))

# Example usage: Assuming 'blog-posts/new_post.md' is the file generated by your code
display_markdown_file('blog-posts/new_post.md')


```markdown
## Electric Cars: Charging Ahead into the Future

The electric car revolution is no longer a distant dream; it's happening right now! With each passing day, EVs are becoming more appealing and accessible to a wider audience. What's driving this electrifying transformation? Let's dive into the top three trends shaping the electric car industry and paving the way for a cleaner, greener future.

One of the most exciting developments is the rapid advancement in battery technology. Forget the limitations of yesterday's batteries! We're talking about next-generation batteries like solid-state and lithium-sulfur, promising longer ranges, faster charging times, and enhanced safety. These innovations directly address the common concerns of potential EV buyers, making electric cars a more practical and attractive option for daily commutes and long road trips alike.

Another crucial factor fueling the EV surge is the ever-expanding charging infrastructure. Imagine a world where finding a charging station is as easy as finding a gas station. That vision is quickly becoming a reality, with a significant increase in public charging stations across the country. This expansion, driven by both government initiatives and private investments, is making EV ownership more convenient and reducing range anxiety, paving the way for mass adoption.

Finally, government incentives and regulations are playing a pivotal role in accelerating the transition to electric mobility. Tax credits and rebates are making EVs more affordable, while stricter emission standards are encouraging automakers to invest in electric vehicle technology. These policies create a favorable environment for EV adoption, driving demand and fostering innovation in the industry. Ready to embrace the electric future? Explore the latest EV models and discover how you can be a part of the movement!
```

# The AI Revolution: Top Trends Shaping the Future
The artificial intelligence industry is experiencing unprecedented growth and innovation, transforming businesses and daily life as we know it. From self-driving cars to virtual assistants, AI's influence is undeniable, and the pace of development shows no signs of slowing down. Staying informed about the latest trends is crucial for anyone looking to understand and leverage the power of AI.

Currently, three major trends are taking center stage. First, advancements in generative AI are democratizing content creation and problem-solving. Second, the growing emphasis on ethical AI and responsible development is ensuring that AI systems are fair, transparent, and beneficial to society. And finally, AI is revolutionizing healthcare, offering new possibilities for diagnostics, treatment, and patient care.

What makes these trends particularly significant is their potential to create a more accessible, trustworthy, and equitable future. Generative AI is empowering non-technical users to harness the power of AI, while ethical AI is building trust and ensuring the long-term sustainability of AI adoption. Moreover, AI's transformative impact on healthcare has the potential to democratize access to quality medical services and improve patient outcomes globally, especially with the convergence of AI and robotics.

Looking ahead, the AI industry promises even more groundbreaking developments. As AI technologies continue to mature, we can expect to see even greater integration into our daily lives, driving innovation and creating new opportunities across various sectors. By embracing these advancements and addressing the ethical considerations that come with them, we can unlock the full potential of AI and create a future where technology benefits all of humanity.

# Creating your own Tools

In [ ]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

class MyToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    student_name: str = Field(..., description="Student name")
    student_roll_no : int = Field(..., description="student id")

class PiaicStudentCard(BaseTool):
    name: str = "Piaic student card generator"
    description: str = "this function will create Piaic student card"
    args_schema: Type[BaseModel] = MyToolInput

    def _run(self, student_name: str, student_roll_no: int ) -> str:
        # Your tool's logic here
        return f"""PIAIC student card
student name: {student_name}
student roll no: {student_roll_no}
UMT, Lahore Pakistan!
        """

In [ ]:
from crewai.tools import tool

@tool("PIAIC fee update")
def my_tool(roll_no: int) -> dict | str:
    """this function search piaic student fee updates, it will required roll no of PIAIC student"""
    #database

    data = {10:'paid',
         20:'unpaid'}


    status = data.get(roll_no)

    if status:
      return {"status": status}
    else:
      return "student not found"

In [ ]:
from crewai import Agent, Task, Crew

card = PiaicStudentCard()


piaic_manager = Agent(
    role="PIAIC manager",
    goal = "Manage all quries regarding PIAIC and you will use only relevant tools for student query",
    backstory="""You are a master at understanding people and their preferences.""",
    tools=[card, my_tool],
    verbose=True,
    llm=llm1
)

piaic_card_creator = Task(
    description="you will be responsible for all PIAIC relevant operations, student query '{query}' you must be know how to answer his question based on final context",
    expected_output="final query answer only",
    agent=piaic_manager
)

crew = Crew(
    agents=[piaic_manager],
    tasks=[piaic_card_creator],
    verbose=True
)

result = crew.kickoff(inputs={
    "query":"I'm PIAIC student my name is Khan Sarwar and my roll number is 10, can you create my student card with student id."
})

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4844f71d-17b4-47fe-9eef-69936fde4c7c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...
    └── 🤖 Agent: PIAIC manager
            Status: In Progress

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student my name is Khan Sarwar and my roll number is 10, can you create my student card with student id.' you must be know how to answer his question based on final context


🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...
    └── 🤖 Agent: PIAIC manager
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...
    └── 🤖 Agent: PIAIC manager
            Status: ✅ Completed
        └── 🔧 Used Piaic student card generator (2)

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Assigned to: PIAIC manager
       Status: ✅ Completed
    └── 🤖 Agent: PIAIC manager
            Status: ✅ Completed
        └── 🔧 Used Piaic student card generator (2)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b74e9ee5-3d1b-4066-939e-6113e5637b8d                                                                     │
│  Agent: PIAIC manager                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4844f71d-17b4-47fe-9eef-69936fde4c7c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

PIAIC student card
student name: Khan Sarwar
student roll no: 10
UMT, Lahore Pakistan!


In [ ]:

result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 10, can you share my fee updates?."
})

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4844f71d-17b4-47fe-9eef-69936fde4c7c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...
    └── 🤖 Agent: PIAIC manager
            Status: In Progress

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 10, can you share my fee updates?.' you must be know how to answer his question based on final context


🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...
    └── 🤖 Agent: PIAIC manager
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...
    └── 🤖 Agent: PIAIC manager
            Status: ✅ Completed
        └── 🔧 Used PIAIC fee update (4)

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Assigned to: PIAIC manager
       Status: ✅ Completed
    └── 🤖 Agent: PIAIC manager
            Status: ✅ Completed
        └── 🔧 Used PIAIC fee update (4)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b74e9ee5-3d1b-4066-939e-6113e5637b8d                                                                     │
│  Agent: PIAIC manager                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4844f71d-17b4-47fe-9eef-69936fde4c7c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

your fee status is paid


In [ ]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 20, can you share my fee updates?."
})

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4844f71d-17b4-47fe-9eef-69936fde4c7c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...
    └── 🤖 Agent: PIAIC manager
            Status: In Progress

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 20, can you share my fee updates?.' you must be know how to answer his question based on final context


🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...
    └── 🤖 Agent: PIAIC manager
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Status: Executing Task...
    ├── 🤖 Agent: PIAIC manager
    │       Status: In Progress
    │   └── 🔧 Used PIAIC fee update (5)
    └── 🤖 Agent: PIAIC manager
            Status: ✅ Completed
        └── 🔧 Used PIAIC fee update (6)

🚀 Crew: crew
└── 📋 Task: b74e9ee5-3d1b-4066-939e-6113e5637b8d
       Assigned to: PIAIC manager
       Status: ✅ Completed
    ├── 🤖 Agent: PIAIC manager
    │       Status: In Progress
    │   └── 🔧 Used PIAIC fee update (5)
    └── 🤖 Agent: PIAIC manager
            Status: ✅ Completed
        └── 🔧 Used PIAIC fee update (6)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b74e9ee5-3d1b-4066-939e-6113e5637b8d                                                                     │
│  Agent: PIAIC manager                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4844f71d-17b4-47fe-9eef-69936fde4c7c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Your fee status is unpaid.


# Process
 * sequential, Herarical

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)



# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts2')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Define your agents
researcher = Agent(
    role="Researcher",
    goal="Conduct thorough research and analysis on AI and AI agents",
    backstory="You're an expert researcher, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently researching for a new client.",
    allow_delegation=False,
    tools=[search_tool,web_rag_tool],
    llm=llm1
)

writer = Agent(
    role="Senior Writer",
    goal="Create compelling content about AI and AI agents",
    backstory="You're a senior writer, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently writing content for a new client.",
    allow_delegation=False,
    tools=[docs_tool,file_tool],
    llm=llm1
)

# Define your task
task = Task(
    description="Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.",
    expected_output="5 bullet points, each with a paragraph and accompanying notes.",
)

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    llm=llm1
)

# Instantiate your crew with a custom manager
crew = Crew(
    agents=[researcher, writer],
    tasks=[task],
    manager_agent=manager,
    function_calling_llm=llm1,
    process=Process.hierarchical,
    verbose=True

)

# Start the crew's work
result = crew.kickoff()
from IPython.display import Markdown

Markdown(result.raw)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 13ffd721-3a24-47e7-b374-5ca08fae53d5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 185f3eb6-3410-42d5-aa93-ace5d7d0d6e3
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 185f3eb6-3410-42d5-aa93-ace5d7d0d6e3
       Status: Executing Task...
    └── 🤖 Agent: Project Manager
            Status: In Progress

# Agent: Project Manager
## Task: Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.


🚀 Crew: crew
└── 📋 Task: 185f3eb6-3410-42d5-aa93-ace5d7d0d6e3
       Status: Executing Task...
    └── 🤖 Agent: Project Manager
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: 185f3eb6-3410-42d5-aa93-ace5d7d0d6e3
       Status: Executing Task...
    ├── 🤖 Agent: Project Manager
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (2)
    ├── 🤖 Agent: Researcher
    │       Status: ✅ Completed
    │   ├── 🔧 Used Search the internet with Serper (6)
    │   └── 🔧 Using Delegate work to coworker (3)
    └── 🤖 Agent: Project Manager
            Status: ✅ Completed
        └── 🔧 Used Read a file's content (3)

🚀 Crew: crew
└── 📋 Task: 185f3eb6-3410-42d5-aa93-ace5d7d0d6e3
       Assigned to: Project Manager
       Status: ✅ Completed
    ├── 🤖 Agent: Project Manager
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (2)
    ├── 🤖 Agent: Researcher
    │       Status: ✅ Completed
    │   ├── 🔧 Used Search the internet with Serper (6)
    │   └── 🔧 Using Delegate work to coworker (3)
    └── 🤖 Agent: Project Manager
            Status: ✅ Completed
        └── 🔧 Used Read a file's content (3)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 185f3eb6-3410-42d5-aa93-ace5d7d0d6e3                                                                     │
│  Agent: Project Manager                                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 13ffd721-3a24-47e7-b374-5ca08fae53d5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

*   **The Unseen Ethical Minefield of AI Agents**

**Paragraph:** As AI agents insinuate themselves into the fabric of our daily routines, automating everything from financial decisions to healthcare recommendations, a Pandora's Box of ethical dilemmas swings open. Are we prepared to grapple with the inherent biases baked into these algorithms, the potential for insidious privacy breaches, and the looming question of accountability when an AI agent makes a life-altering mistake? This article delves into the murky depths of AI ethics, exposing the hidden dangers lurking beneath the surface of seemingly benign automation.

**Notes:**
*   **Angle:** Expose the potential negative consequences and ethical challenges of AI agents. Focus on the risks that aren't immediately obvious.
*   **Target Audience:** General public, policymakers, ethicists, and anyone concerned about the societal impact of AI.

*   **AI Agents as the Ultimate Productivity Hack: A Day in the Life**

**Paragraph:** Imagine a world where your schedule manages itself, your inbox is meticulously curated, and your to-do list magically shrinks before your eyes. This isn't science fiction; it's the reality achievable with AI agents. This article provides a practical, step-by-step guide to integrating AI agents into your daily workflow, transforming you from a reactive firefighter into a proactive productivity powerhouse. Discover the tools, techniques, and strategies to harness the power of AI and reclaim your time, energy, and focus.

**Notes:**
*   **Angle:** Provide a practical, actionable guide to using AI agents for productivity. Focus on specific tools and techniques that readers can implement immediately.
*   **Target Audience:** Professionals, entrepreneurs, students, and anyone looking to improve their productivity and time management skills.

*   **The Generative AI Arms Race: Will Open Source Models Democratize AI Agent Development?**

**Paragraph:** The race to develop the most powerful generative AI models is heating up, with tech giants vying for dominance. But will this AI arms race lead to a future where only a select few control the technology? This article explores the critical role of open-source models in democratizing AI agent development, empowering individuals and small teams to create innovative AI-powered solutions. We'll examine the leading open-source projects, analyze their potential to disrupt the industry, and consider the implications for the future of AI innovation.

**Notes:**
*   **Angle:** Analyze the competitive landscape of generative AI and its impact on the accessibility and democratization of AI agent development. Focus on the role of open-source models.
*   **Target Audience:** Developers, researchers, investors, and anyone interested in the future of AI and the competitive dynamics of the tech industry.

*   **Beyond Automation: How AI Agents are Fostering Human Creativity and Innovation**

**Paragraph:** Forget the dystopian visions of robots replacing artists. AI agents are emerging as powerful collaborators, unlocking new realms of human creativity and innovation. From AI-powered music composition tools to generative art platforms, this article explores the unexpected ways AI is augmenting human capabilities, fostering new forms of artistic expression, and pushing the boundaries of what's possible. Discover how AI agents are not just automating tasks, but inspiring a new era of human-AI co-creation.

**Notes:**
*   **Angle:** Highlight the positive impact of AI agents on human creativity and innovation. Focus on examples of how AI is augmenting human capabilities in artistic and creative fields.
*   **Target Audience:** Artists, designers, musicians, innovators, and anyone interested in the intersection of AI and creativity.

*   **The AI Agent Revolution in Healthcare: Promise and Peril**

**Paragraph:** AI agents are poised to revolutionize healthcare, offering the potential to improve diagnoses, personalize treatments, and streamline administrative processes. But alongside this incredible promise lies a complex web of risks, including data privacy concerns, algorithmic bias, and the potential for misdiagnosis. This article delves into the transformative potential of AI agents in healthcare, while carefully examining the ethical and practical challenges that must be addressed to ensure safe and equitable deployment.

**Notes:**
*   **Angle:** Provide a balanced overview of the potential benefits and risks of using AI agents in healthcare. Focus on both the promise and the peril.
*   **Target Audience:** Healthcare professionals, patients, policymakers, and anyone interested in the future of healthcare and the role of AI.

# Advance Tools

In [ ]:
from pydantic import BaseModel, Field

class TeacherAssignmentOutPut(BaseModel):
  score :int  = Field(..., description="student assignment score")
  feedback : str = Field(..., description="teacher feedback")
  question : str = Field(..., description="question")
  answer : str = Field(..., description="python code for markdown format")

In [ ]:
from crewai import Agent
from crewai_tools import CodeInterpreterTool
from crewai import Task, Crew, Process

teacher = Agent(
    role="AI teacher",
    goal="you are Agentic AI teacher you have to check python assignment code submission.",
    backstory="you have to check student python code submission.",
    tools=[CodeInterpreterTool()],
    llm=llm1
)

assignment_check = Task(
    description="you are Agentic AI teacher you have to check python assignment code submission. question: '''{qestion}''' student solution : '''{solution}''' ",
    expected_output="final code is runing, assign number between 1-10",
    output_pydantic=TeacherAssignmentOutPut,
    agent=teacher # Added the missing agent here
)


crew = Crew(
    agents=[teacher],
    tasks=[assignment_check],
    process=Process.sequential,
    verbose=True,
)

question = "create function add two numbers"

solution = """
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

"""

result = crew.kickoff(inputs={
    "qestion": question,
    "solution" : solution
})

print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 275dbafb-6bee-4202-bc77-eb884802d05a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: bf7835af-14b9-4b62-bae2-2f8d16e37a29
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: bf7835af-14b9-4b62-bae2-2f8d16e37a29
       Status: Executing Task...
    └── 🤖 Agent: AI teacher
            Status: In Progress

# Agent: AI teacher
## Task: you are Agentic AI teacher you have to check python assignment code submission. question: '''create function add two numbers''' student solution : '''
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

''' 


🚀 Crew: crew
└── 📋 Task: bf7835af-14b9-4b62-bae2-2f8d16e37a29
       Status: Executing Task...
    └── 🤖 Agent: AI teacher
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: bf7835af-14b9-4b62-bae2-2f8d16e37a29
       Status: Executing Task...
    └── 🤖 Agent: AI teacher
            Status: ✅ Completed
        └── 🔧 Failed Code Interpreter (2)

🚀 Crew: crew
└── 📋 Task: bf7835af-14b9-4b62-bae2-2f8d16e37a29
       Assigned to: AI teacher
       Status: ✅ Completed
    └── 🤖 Agent: AI teacher
            Status: ✅ Completed
        └── 🔧 Failed Code Interpreter (2)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bf7835af-14b9-4b62-bae2-2f8d16e37a29                                                                     │
│  Agent: AI teacher                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 275dbafb-6bee-4202-bc77-eb884802d05a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

score=9 feedback="The code is correct and well-formatted. The function `add_two_number` correctly adds two numbers. The type hints are a good addition. The function name could be slightly improved to `add_numbers` for better readability, as it can handle more than just 'two' numbers even though the prompt asked specifically for two. Consider adding a docstring to explain what the function does." question='create function add two numbers' answer='def add_two_number(num1 : int , num2 : int):\n  return num1 + num2'


# Collabration

In [ ]:
from crewai import Agent, Task, Workflow

class StudentDatabase:
    def __init__(self):
        self.students = {}

    def add_student(self, student_id, name, age, grade):
        self.students[student_id] = {'name': name, 'age': age, 'grade': grade}
        return f"Student {name} added successfully."

    def get_student(self, student_id):
        return self.students.get(student_id, "Student not found.")

    def update_student(self, student_id, name=None, age=None, grade=None):
        if student_id in self.students:
            if name:
                self.students[student_id]['name'] = name
            if age:
                self.students[student_id]['age'] = age
            if grade:
                self.students[student_id]['grade'] = grade
            return f"Student {student_id} updated successfully."
        return "Student not found."

    def delete_student(self, student_id):
        if student_id in self.students:
            del self.students[student_id]
            return f"Student {student_id} deleted successfully."
        return "Student not found."

class StudentManagementAgent:
    def __init__(self):
        self.db = StudentDatabase()

    def perform_task(self, action, **kwargs):
        if action == 'add':
            return self.db.add_student(kwargs['student_id'], kwargs['name'], kwargs['age'], kwargs['grade'])
        elif action == 'get':
            return self.db.get_student(kwargs['student_id'])
        elif action == 'update':
            return self.db.update_student(kwargs['student_id'], kwargs.get('name'), kwargs.get('age'), kwargs.get('grade'))
        elif action == 'delete':
            return self.db.delete_student(kwargs['student_id'])
        else:
            return "Invalid action."

if __name__ == "__main__":
    agent = StudentManagementAgent()
    print(agent.perform_task('add', student_id=1, name='John Doe', age=20, grade='A'))
    print(agent.perform_task('get', student_id=1))
    print(agent.perform_task('update', student_id=1, age=21))
    print(agent.perform_task('delete', student_id=1))


ImportError: cannot import name 'Workflow' from 'crewai' (/usr/local/lib/python3.11/dist-packages/crewai/__init__.py)